In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import numpy as np
import os

module_path = os.path.abspath(os.path.join(os.getcwd() + "/.."))
if module_path not in sys.path:
    sys.path.append(module_path)

from ad_afqmc import driver, pyscf_interface, mpi_jax

from pyscf import fci, gto, scf

np.set_printoptions(precision=5, suppress=True)

### RHF trial ph-afqmc


In [ ]:
r = 1.012
theta = 106.67 * np.pi / 180.0
rz = r * np.sqrt(np.cos(theta / 2) ** 2 - np.sin(theta / 2) ** 2 / 3)
dc = 2 * r * np.sin(theta / 2) / np.sqrt(3)
atomstring = f"""
                 N 0. 0. 0.
                 H 0. {dc} {rz}
                 H {r * np.sin(theta/2)} {-dc/2} {rz}
                 H {-r * np.sin(theta/2)} {-dc/2} {rz}
              """
mol = gto.M(atom=atomstring, basis="sto-6g", verbose=3, symmetry=0)
mf = scf.RHF(mol)
mf.kernel()

# fci
cisolver = fci.FCI(mol, mf.mo_coeff)
efci, ci = cisolver.kernel()
print(f"FCI energy: {efci}")

pyscf_interface.prep_afqmc(mf)
options = {
    "dt": 0.005,
    "n_eql": 3,
    "n_ene_blocks": 1,
    "n_sr_blocks": 5,
    "n_blocks": 100,
    "n_walkers": 50,
    "seed": 98,
    "walker_type": "rhf",
    "trial": "rhf",
}
e_afqmc, err_afqmc = driver.afqmc(*(mpi_jax._prep_afqmc(options)))

### UHF trial ph-afqmc


In [ ]:
r = 2.0
nH = 6
atomstring = ""
for i in range(nH):
    atomstring += "H 0 0 %g\n" % (i * r)
mol = gto.M(atom=atomstring, basis="sto-6g", verbose=3, unit="bohr")
mf = scf.RHF(mol)
mf.kernel()

umf = scf.UHF(mol)
umf.kernel()
mo1 = umf.stability(external=True)[0]
umf = umf.newton().run(mo1, umf.mo_occ)

# fci
cisolver = fci.FCI(mol, mf.mo_coeff)
efci, ci = cisolver.kernel()
print(f"FCI energy: {efci}")

pyscf_interface.prep_afqmc(umf)
options = {
    "dt": 0.005,
    "n_eql": 3,
    "n_ene_blocks": 1,
    "n_sr_blocks": 5,
    "n_blocks": 100,
    "n_walkers": 50,
    "seed": 98,
    "walker_type": "rhf",
    "trial": "uhf",
}
e_afqmc, err_afqmc = driver.afqmc(*(mpi_jax._prep_afqmc(options)))